In [84]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [85]:
df = pd.read_csv("seattle-weather.csv")

In [86]:
df.index = pd.to_datetime(df.date)

In [87]:
df.drop(['date'], axis=1, inplace=True)
df.drop(['temp_max'], axis=1, inplace=True)
df.drop(['temp_min'], axis=1, inplace=True)

In [88]:
df.head()

,temp
date,
2012-01-01,8.90
2012-01-02,6.70
2012-01-03,9.45
2012-01-04,8.90
2012-01-05,5.85


In [89]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

In [90]:
sequence_length = 10 
num_features = len(df.columns)

In [91]:
scaled_data

array([[0.41639344],
       [0.3442623 ],
       [0.43442623],
       ...,
       [0.30655738],
       [0.32459016],
       [0.26229508]])

In [92]:
sequences = []
labels = []
for i in range(len(scaled_data) - sequence_length):
    seq = scaled_data[i:i+sequence_length]
    label = scaled_data[i+sequence_length][0] 
    sequences.append(seq)
    labels.append(label)

In [93]:
seq

array([[0.35081967],
       [0.36229508],
       [0.35245902],
       [0.28032787],
       [0.25245902],
       [0.2704918 ],
       [0.29836066],
       [0.34262295],
       [0.30655738],
       [0.32459016]])

In [94]:
sequences = np.array(sequences)
labels = np.array(labels)

In [95]:
train_size = int(0.8 * len(sequences))
train_x, test_x = sequences[:train_size], sequences[train_size:]
train_y, test_y = labels[:train_size], labels[train_size:]

print("Train X shape:", train_x.shape)
print("Train Y shape:", train_y.shape)
print("Test X shape:", test_x.shape)
print("Test Y shape:", test_y.shape)

Train X shape: (1152, 10, 1)
Train Y shape: (1152,)
Test X shape: (289, 10, 1)
Test Y shape: (289,)


In [96]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping


model = Sequential()
model.add(LSTM(units=100, input_shape=(10,1)))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')

In [97]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(
    train_x, train_y,
    epochs=100,
    batch_size=64,
    validation_split=0.2,  # Use part of the training data as validation
    callbacks=[early_stopping]
)

Epoch 1/100
15/15 [==============================] - 4s 63ms/step - loss: 0.0487 - val_loss: 0.0102
Epoch 2/100
15/15 [==============================] - 0s 19ms/step - loss: 0.0124 - val_loss: 0.0093
Epoch 3/100
15/15 [==============================] - 0s 20ms/step - loss: 0.0099 - val_loss: 0.0095
Epoch 4/100
15/15 [==============================] - 0s 20ms/step - loss: 0.0089 - val_loss: 0.0094
Epoch 5/100
15/15 [==============================] - 0s 20ms/step - loss: 0.0088 - val_loss: 0.0089
Epoch 6/100
15/15 [==============================] - 0s 30ms/step - loss: 0.0085 - val_loss: 0.0088
Epoch 7/100
15/15 [==============================] - 0s 21ms/step - loss: 0.0084 - val_loss: 0.0087
Epoch 8/100
15/15 [==============================] - 0s 22ms/step - loss: 0.0084 - val_loss: 0.0089
Epoch 9/100
15/15 [==============================] - 0s 22ms/step - loss: 0.0087 - val_loss: 0.0087
Epoch 10/100
15/15 [==============================] - 0s 21ms/step - loss: 0.0084 - val_loss: 0.0085

In [100]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

predictions = model.predict(test_x)

mae = mean_absolute_error(test_y, predictions)
mse = mean_squared_error(test_y, predictions)
rmse = np.sqrt(mse)
r_squared = r2_score(test_y, predictions)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2 score):", r_squared)

10/10 [==============================] - 0s 5ms/step
Mean Absolute Error (MAE): 0.048979105913528845
Mean Squared Error (MSE): 0.003913267947790456
Root Mean Squared Error (RMSE): 0.06255611838813575
R-squared (R2 score): 0.8827588915715487
